In [1]:
import requests
import pandas as pd

In [2]:
API_URL = 'https://apis.datos.gob.ar/series/api/series/?ids=149.1_SOR_PRIADO_OCTU_0_25'
response = requests.get(API_URL)
data = response.json()

In [3]:
# We make the transformations. Converting the json to dataframe and then converting the date column to datetime.
df = pd.DataFrame(data["data"], columns=["Date", "Value"])
df["Date"] = pd.to_datetime(df["Date"])
df.dtypes

Date     datetime64[ns]
Value           float64
dtype: object

In [4]:
# Importing neccesary libraries and initiating connection
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

conn_string = "host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port=5439 dbname=data-engineer-database user=leo_carulli_coderhouse password=PT3o7PK1Rh"
conn = psycopg2.connect(conn_string)

In [5]:
# Create the SQLAlchemy engine
engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)

In [6]:
# Create the table if it doesn't exist
create_table_query = """
    CREATE TABLE IF NOT EXISTS TablaDeSalarios (
        fecha DATE DISTKEY,
        indice_de_salarios NUMERIC
    )
    SORTKEY (fecha);
"""
with conn.cursor() as cursor:
    cursor.execute(create_table_query)
    conn.commit()

In [7]:
# Convert the DataFrame to a temporary table
df.to_sql('temp_table', engine, index=False, if_exists='replace')

In [8]:
# Execute the TRUNCATE statement. This will delete all prior information present on the database
with conn.cursor() as cursor:
    query = "TRUNCATE TABLE TablaDeSalarios;"
    cursor.execute(query)
    conn.commit()
    print("All rows deleted successfully.")


All rows deleted successfully.


In [9]:
# Execute an SQL query to insert data into the target table
with conn.cursor() as cursor:
    query = "INSERT INTO TablaDeSalarios (fecha, indice_de_salarios) SELECT Date, Value FROM temp_table;"
    cursor.execute(query)
    conn.commit()

In [10]:
# Execute the query to check results
with conn.cursor() as cursor:
    query = "SELECT * FROM TablaDeSalarios ORDER BY fecha DESC;"
    cursor.execute(query)
    result = cursor.fetchall()
    # Process the query result as needed
    for row in result:
        print(row)

(datetime.date(2023, 3, 1), 1202.1)
(datetime.date(2023, 2, 1), 1114.3)
(datetime.date(2023, 1, 1), 1042.4)
(datetime.date(2022, 12, 1), 996.6)
(datetime.date(2022, 11, 1), 953.34)
(datetime.date(2022, 10, 1), 888.1)
(datetime.date(2022, 9, 1), 843.4)
(datetime.date(2022, 8, 1), 797.04)
(datetime.date(2022, 7, 1), 737.01)
(datetime.date(2022, 6, 1), 699.36)
(datetime.date(2022, 5, 1), 665.7)
(datetime.date(2022, 4, 1), 622.3)
(datetime.date(2022, 3, 1), 589.2)
(datetime.date(2022, 2, 1), 559.0)
(datetime.date(2022, 1, 1), 538.2)
(datetime.date(2021, 12, 1), 514.3)
(datetime.date(2021, 11, 1), 507.0)
(datetime.date(2021, 10, 1), 485.29)
(datetime.date(2021, 9, 1), 467.91)
(datetime.date(2021, 8, 1), 451.64)
(datetime.date(2021, 7, 1), 437.0)
(datetime.date(2021, 6, 1), 415.6)
(datetime.date(2021, 5, 1), 408.4)
(datetime.date(2021, 4, 1), 394.54)
(datetime.date(2021, 3, 1), 376.68)
(datetime.date(2021, 2, 1), 362.33)
(datetime.date(2021, 1, 1), 344.61)
(datetime.date(2020, 12, 1), 331.26

In [ ]:
# Close the connection
conn.close()